# <p style="text-align: center;"> Relationship Analysis of People’s Income and Restaurants Nearby </p>
### <p style="text-align: center;"> Liang Zhang, Shan Li </p>
### <p style="text-align: center;"> School of Computer Science, Carnegie Mellon University </p>

## Background
Does local individual income level and other financial information contribute into the success of local restaurants? Where should one run a restaurant business in US? We are going to examine the nationwide data of the popularity of restaurants from Yelp and local individual income and other financial information from Internal Revenue Service, explore the relation of both features within each of them and also features cross these two fields, and try to answer the questions aforementioned. Yelp is a widely used website that provides crowd-sourced reviews about local businesses, and the Internal Revenue Service is the nation's tax collection agency and administers the Internal Revenue Code enacted by Congress.$^{[1]}$

## Data Resource
### SOI Data from IRS
ZIP Code data show selected income and tax items classified by State, ZIP Code, and size of adjusted gross income. Data are based on individual income tax returns filed with the IRS and are available for Tax Years 1998, 2001, and 2004 through 2015. The data include items, such as: number of returns, which approximates the number of households; number of personal exemptions, which approximates the population; adjusted gross income; wages and salaries; dividends before exclusion; interest received and so on$^{[2]}$.

In our work, we only employed SOI data for Tax Years 2015, which is the latest recorded one. Also, the SOI data is rich in information which includes tens of thousands of tax-related information. We applied only a small part of this data for our use. More specifically, we only employed "$\textit{total income}$", "$\textit{salary}$", "$\textit{business income}$", "$\textit{unemployment compensation}$", "$\textit{pension}$" and "$\textit{capital gain}$" these six income-related information.
### Yelp Data
We use both crawler and Yelp Fusion API in this project. 

For raw crawler, we only retrieved the oldest date of review appear in the website page provided by Yelp.

For other basic information of restaurant, we employed Yelp's Fusion API$^{[3]}$ to get thiese feature.

In [4]:
# setup library imports
import io, time, json
import requests
import pickle
import csv
import re
from bs4 import BeautifulSoup

## Data Crawling and Preprocessing

## SOI data cleaning
* $\textbf{Inner join related items}$:  
The SOI data contains 6 records for each postal code and each records contains all the income-related information that we need. The 6 records are for diffrent level of "$\textit{total income}$" which is one of the attributes it the records. Since we need to use averaged value of all features in one region of a specific postal code, we need to join the 6 records together by adding all income values togeter within one specific postal code.


* $\textbf{Extract 6 income-related feature for each zip code}$:  
The SOI data contains 130 features for each record, however, we found that there are only 6 features draw our interest. So we choose these 6 feature subjectively by filtering out all other features.


* $\textbf{Drop any item that lack of any features or has number of returns less than a specified threshold to clean data}$:  
The SOI data contains lots of noise that needs to be taken care of such as lack of features, short of number of returns (which make the records) and so on. Basically, we set a threshold of 2500, and we simply drop the entire record in which any feature that has the number of returns less than the threshold.

In [ ]:
def preprocess_income_data(filename):
    threshold = 1200
    dic = {'zip_code': [], 'total_income':[], 'salary':[], 'business_income':[], 'unemployment_compensation':[], 'pension':[], 'capital_gain':[]}
    dic_res = {'zip_code': [], 'total_income':[], 'salary':[], 'business_income':[], 'unemployment_compensation':[], 'pension':[], 'capital_gain':[]}
    with open(filename, encoding = 'utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['zipcode'] == '99999' or row['zipcode'] == '0':
                continue
            if len(dic['zip_code']) == 0 or row['zipcode'] != dic['zip_code'][-1]:
                if len(dic['zip_code']) != 0 and (dic['total_income'][-1] < threshold or dic['salary'][-1] < threshold or dic['business_income'][-1] < threshold or dic['unemployment_compensation'][-1] < threshold or dic['pension'][-1] < threshold or dic['capital_gain'][-1] < threshold):
                    dic['zip_code'][-1] = row['zipcode']
                    dic['total_income'][-1] = int(row['N02650'])
                    dic['salary'][-1] = int(row['N00200'])
                    dic['business_income'][-1] = int(row['N00900'])
                    dic['unemployment_compensation'][-1] = int(row['N02300'])
                    dic['pension'][-1] = int(row['N01700'])
                    dic['capital_gain'][-1] = int(row['N01000'])
                    dic_res['zip_code'][-1] = row['zipcode']
                    dic_res['total_income'][-1] = int(row['A02650'])
                    dic_res['salary'][-1] = int(row['A00200'])
                    dic_res['business_income'][-1] = int(row['A00900'])
                    dic_res['unemployment_compensation'][-1] = int(row['A02300'])
                    dic_res['pension'][-1] = int(row['A01700'])
                    dic_res['capital_gain'][-1] = int(row['A01000'])
                else:
                    dic['zip_code'].append(row['zipcode'])
                    dic['total_income'].append(int(row['N02650']))
                    dic['salary'].append(int(row['N00200']))
                    dic['business_income'].append(int(row['N00900']))
                    dic['unemployment_compensation'].append(int(row['N02300']))
                    dic['pension'].append(int(row['N01700']))
                    dic['capital_gain'].append(int(row['N01000']))
                    dic_res['zip_code'].append(row['zipcode'])
                    dic_res['total_income'].append(int(row['A02650']))
                    dic_res['salary'].append(int(row['A00200']))
                    dic_res['business_income'].append(int(row['A00900']))
                    dic_res['unemployment_compensation'].append(int(row['A02300']))
                    dic_res['pension'].append(int(row['A01700']))
                    dic_res['capital_gain'].append(int(row['A01000']))
            else:
                dic['total_income'][-1] += int(row['N02650'])
                dic['salary'][-1] += int(row['N00200'])
                dic['business_income'][-1] += int(row['N00900'])
                dic['unemployment_compensation'][-1] += int(row['N02300'])
                dic['pension'][-1] += int(row['N01700'])
                dic['capital_gain'][-1] += int(row['N01000'])
                dic_res['total_income'][-1] += int(row['A02650'])
                dic_res['salary'][-1] += int(row['A00200'])
                dic_res['business_income'][-1] += int(row['A00900'])
                dic_res['unemployment_compensation'][-1] += int(row['A02300'])
                dic_res['pension'][-1] += int(row['A01700'])
                dic_res['capital_gain'][-1] += int(row['A01000'])
        if len(dic['zip_code']) != 0 and (dic['total_income'][-1] < threshold or dic['salary'][-1] < threshold or dic['business_income'][-1] < threshold or dic['unemployment_compensation'][-1] < threshold or dic['pension'][-1] < threshold or dic['capital_gain'][-1] < threshold):
            dic['zip_code'] = dic['zip_code'][:-1]
            dic['total_income'] = dic['total_income'][:-1]
            dic['salary'] = dic['salary'][:-1]
            dic['business_income'] = dic['business_income'][:-1]
            dic['unemployment_compensation'] = dic['unemployment_compensation'][:-1]
            dic['pension'] = dic['pension'][:-1]
            dic['capital_gain'] = dic['capital_gain'][:-1]
            dic_res['zip_code'] = dic_res['zip_code'][:-1]
            dic_res['total_income'] = dic_res['total_income'][:-1]
            dic_res['salary'] = dic_res['salary'][:-1]
            dic_res['business_income'] = dic_res['business_income'][:-1]
            dic_res['unemployment_compensation'] = dic_res['unemployment_compensation'][:-1]
            dic_res['pension'] = dic_res['pension'][:-1]
            dic_res['capital_gain'] = dic_res['capital_gain'][:-1]
    return dic_res

In [81]:
income_dict = preprocess_income_data('15zpallagi.csv')
print(income_dict['zip_code'])
print(len(income_dict['zip_code']))
print(income_dict)

# with open('preprocessed_income_data_2015.pickle', 'wb') as file:
#     pickle.dump(income_dict, file, protocol=pickle.HIGHEST_PROTOCOL)

# with open('preprocessed_income_data_2015.pickle', 'rb') as file:
#     income_dict = pickle.load(file)

['99507', '92336', '94112', '95076']
4
{'zip_code': ['99507', '92336', '94112', '95076'], 'total_income': [19570, 40660, 44190, 37210], 'salary': [17150, 36290, 37490, 32880], 'business_income': [2700, 7380, 8530, 4540], 'unemployment_compensation': [4700, 2620, 2600, 6120], 'pension': [2980, 4700, 5410, 4150], 'capital_gain': [2850, 2510, 5930, 3300]}


### Visualize the zip code we got from preprocessed SOI data

Figure 1 shows the distribution of postal codes we got from our preprocessed SOI data.


As is shown, postal codes aggregated around areas that has large citys such as New York City and Carlifornia. Since we filtered out postal codes where there are less people report their income information, it reflects that large cities contains more people that afford tax and the corrsponding income of theese regions are higher than other areas.

<img src="Map_US.jpg" alt="US_MAP">
<p style="text-align: center;">Figure 1. Distributon of our yelp data on US map</p>

## Yelp data crawling and preprocessing

### Yelp data crawling
* Using Yelp Fusion API, we got all restaurant basic information including “$\textit{name}$”, “$\textit{rating}$”, “$\textit{review count}$”, “$\textit{price}$”.
* In the previous step, we preprocessed the SOI data and finally got a a list of 8000+ records for each of which we have an unique postal code corresponding to it. In this step, we search via Yelp's API given the list of zipcode so that we cound connect data from SOI data and that from Yelp data.

In [3]:
def read_api_key(filepath):
    with open(filepath, 'r') as f:
        api_key = f.read().replace('\n','')
    return api_key

In [6]:
def all_restaurants(api_key, zip_code):
    # find restaurants established before 2015 according to zip_code
    # zip_code: str
    offset = 0
    headers = {
        "authorization": 'Bearer %s' % api_key, # for yelp API
    }
    total_num = 1
    dic_list = []
    while offset < total_num:
        params = {
            "location": zip_code,
            "categories": "restaurant",
            "offset": offset,
            "limit": 50,
            "sort_by": "review_count",
        }
        result = requests.get('https://api.yelp.com/v3/businesses/search', headers= headers, params=params).json()
        if 'businesses' not in result:
            break
        dic_list += result['businesses']
        total_num = result['total']
        offset += 50
        time.sleep(0.2)
    return dic_list


In [15]:
def get_data_from_yelp(zipcode):
    data = all_restaurants(read_api_key('api_key.txt'), zipcode)
    
    res = []
    for data_i in data:
        if 'name' not in data_i \
        or 'rating' not in data_i \
        or 'review_count' not in data_i \
        or 'price' not in data_i \
        or 'transactions' not in data_i:
            continue

        cur_list = [zipcode, data_i['name'], data_i['rating'], data_i['review_count'], data_i['price'], str(data_i['transactions'])]
#         print(cur_list)
        res.append(cur_list)
    print("final length is: ", len(res))
    return res

### Yelp data preprocessing
* $\textbf{Remove the "$\textit{noise restaurants}$"}$:  
Since the SOI data we got from RSI is only for Tax Year 2015, we want to filter all information to the restaurants that happens after 2015. However, can not find a perfect way to do this. For example, we have no information which part of rate to a specific restaurant comes before or after 2015. So we here used a working around solution: when we find a specific restaurant via the Yelp's search, we will go into their website and found the oldest date among all reviews of this restaurant. Next, we simply drop the restaurant if the date is after 2015, and this restaurant was treated as "$\textit{noise restaurants}$". Although it is not a perfect approach, in this way we could slightly reduce the noice of our data dataset.


* $\textbf{Drop restaurants that have any blank attribute basic information above}$:  
As a general approach, we simply drop the retrieved restaurants that has any of features we are looking at empty.


* $\textbf{Sort the data by review count}$:  
One of the limt of the Yelp Fusion API is it constrains the max number of access of its business to 1000. However, we found that most of the number of restaurants given a specific region is less than 1000. And in order to further reduce the noise of data, we sort the Yelp data by its "$\textit{review count}$" attribute, which means we only retrieve the top 1000 popular restaurants for those regions that has a number of restaurants larger than 1000. 

In [4]:
def check_restaurant_before_2015(url):
    """
    Check whether or not the restaurant established before 2015.

    Parameters:
        url (string): Yelp URL corresponding to the restaurant of interest.

    Returns:
        (boolean): whether or not the restaurant established before 2015.
    """
    url += "&sort_by=date_asc"
    html = requests.get(url).content
    url=None
    while True:
        soup = BeautifulSoup(html, 'html.parser')
        for div in soup.find_all("div", class_="review review--with-sidebar"):
            tmp = re.search("[0-9]*/[0-9]*/[0-9]*", div.find("span", class_='rating-qualifier').get_text())
            if tmp is not None:
                tmp = tmp.group(0).rstrip()
                tmp = (tmp[tmp.rfind('/') + 1:])
                try:
                    if int(tmp) <= 2015:
                        return True
                    return False
                except:
                    continue
        url=soup.find("a", class_="u-decoration-none next pagination-links_anchor")
        if url is not None:
            url=url['href']
            html=requests.get(url).content
        else:
            break
    return False

### SQLite database  
We use SQLite as our database and store all data in a "$\textit{.db}$" file to manage our crawled data. We commit all changes to our database for each record we crawled.

In [ ]:
import csv, sqlite3

con = sqlite3.connect("yelp.db")
cur = con.cursor()
cur.execute("CREATE TABLE yelp (zipcode TEXT,name TEXT,rating REAL,review_count INT,price TEXT,transactions TEXT)")

with open("zipcode.txt") as f: 
    zipcode = f.read().splitlines()
for i, zc in enumerate(zipcode):
    if i <= 458:
        continue
    print(i)
    print(zc)
    res = get_data_from_yelp(zc)

    cur.executemany("INSERT INTO yelp (zipcode,name,rating,review_count,price,transactions) VALUES (?, ?, ?, ?, ?, ?);", res)
    con.commit()
con.close()

## Relation Analysis
For all following figures that we used to illustrate our work, we use "$\textit{RStudio}$" to plot our figures which is base on R language.

### Distribution and Correlation for SOI data
We store SOI data into our SQLite database also. To extract SOI data we use the following SQL string:

In [ ]:
sql_string = 'CREATE TABLE irs_1 AS\
  SELECT \
  	STATE, zipcode, Ntotal_return, Atotal_return/Ntotal_return as Avg_total_return, \
  	Nsalary, Asalary/Nsalary as Avg_salary,\
  	Nbusiness, Abusiness/Nbusiness as Avg_business,\
  	Nunemploy, Aunemploy/Nunemploy as Avg_unemploy,\
  	Npension, Apension/Npension as Avg_pension,\
  	Ncapital, Acapital/Ncapital as Avg_capital\
  FROM (\
	SELECT STATE, zipcode, sum(N02650) as Ntotal_return, sum(A02650) as Atotal_return,\
    sum(N00200) as Nsalary, sum(A00200) as Asalary, sum(N00900) as Nbusiness,\
    sum(A00900) as Abusiness, sum(N02300) as Nunemploy, sum(A02300) as Aunemploy,\
    sum(N01700) as Npension, sum(A01700) as Apension, sum(N01000) as Ncapital,\
    sum(A01000) as Acapital FROM irs GROUP BY zipcode\
	) where Ntotal_return >= 1200 and Nsalary >= 1200 and Nbusiness >= 1200 and Nunemploy >= 1200 and Npension >= 1200 and Ncapital >= 1200;'

* $\textbf{Distribution for one feature itself in SOI data}$  
   In order to visulize the data distribution of SOI data itself, we examined the cumulative distribution for people's total income, as well as other forms of incomes, where salary takes the majority of the total income. Figure 2 shows the overview of distribution of SOI data. There are 6 attributes/featuers we extracted form SOI data, and for each sub-graph in Figure 6, we could see a distribution for one attribute/feature in SOI data.

<img src="Overview of distribution of SOI data.jpg" alt="SOI_data">
<p style="text-align: center;">Figure 2. Overview of distribution of SOI data</p>

* $\textbf{Relation for two features in SOI data using Linear Regression}$  
   We examined the correlation between average salary and other kind of incomes in SOI data. Therefore we run Linear Regression and pot the figure out in order to visulize the relaton between some pair of features in SOI data. Figure 3 shows these relation below.  
   As Figure 3 shows, the business income and average pension are positively related to average salary while Capital gain seems quite independent, which is not surprising.

<img src="SOI.jpg" alt="SOI_relation_data">
<p style="text-align: center;">Figure 3. Linear Relation between features in SOI data</p>

### Distribution and Correlation for Yelp data
We store Yelp data into our SQLite database. To extract Yelp data we use the following SQL string:

In [ ]:
yelp_mean_sql_string = 'CREATE TABLE yelp_mean AS\
                        SELECT zipcode, avg(rating) as avg_rating,\
                        avg(review_count) as review_count\
                        FROM yelp\
                        GROUP BY zipcode;'

yelp_price_sql_string = 'CREATE TABLE yelp_price AS\
                        SELECT zipcode, avg(price_val) as avgprice\
                        FROM\
                        (SELECT zipcode, length(price) as price_val\
                        FROM yelp)\
                        GROUP BY zipcode;'

* $\textbf{Distribution for one feature itself in Yelp data}$  
   In order to visulize the data distribution of Yelp data itself, we examined the cumulative distribution for each one of three features in Yelp data. Figure 4 shows the overview of distribution of Yelp data.  
   From Yelp Data, we can see the average rating is evenly distributed, while over 80% of restuarants has fewer than 200 reviews and the top 2% of them has over a thousand reviews. 

<img src="Overview of distribution of Yelp data.jpg" alt="Yelp_data">
<p style="text-align: center;">Figure 4. Overview of distribution of Yelp data</p>

* $\textbf{Relation for two features in Yelp data using Linear Regression}$  
   Figure 5 shows linear relations between any two pairs of features in Yelp data. We do find that the average rating tends to be higher when the price goes up. On the other hand, there are still a bunch of high-rating restaurants with moderate price. The plot on the right side of Figure 5 shows that when there are many reviews for a restuarant, the rating is very likely to be high.

<img src="Yelp.jpg" alt="Yelp_data">
<p style="text-align: center;">Figure 5. Linear Relation between features in Yelp data</p>

### Correlation between two different data set
We also examined and visulized the correlation between SOI and Yelp data set. For SOI data, we only select "$\textit{Average Salary}$" for a region from SOI; for Yelp data, we examined all three features including "$\textit{Average Price}$", "$\textit{Averaged Review Count}$" and "$\textit{Averaged Rating}$".

When did this by joining the two tables by the same zipcode. As Figure 6 shows, the correlation between local peoples income and the rating is actually weak. This means the key to a successful restaurant is NOT the wealthiness of the people in this area!

<img src="Yelp&SOI.jpg" alt="Yelp_data">
<p style="text-align: center;">Figure 6. Linear Relation between features in SOI and Yelp data</p>

## Conclusion
We focused on the relation analysis of data in SOI and Yelp data set. We crawled and preprocessed data from RSI website and Yelp website; visualized the individual data distribution for each data set; explore the correlation of pair of features both within each data set and also cross two data set. Finally we could answer the question that we put forward at the begining: Indeed there are relations between features corresponding to one restaurant like number of reviews and its rating, and also there are relations between local people's average salary and averged review count of local restaurant, however, local individual income level and other financial information barely contribute into the success of local restaurants (as Figure 6 suggests), which means restaurant runners should not focus on select address according to outer factors like average salaries in one specific region, instead, they may should mind their own business and may be only improve the quality of food and service themself will lead them to success in the end.

# Reference
[1]	About IRS, https://www.irs.gov/about-irs

[2]	SOI Tax Stats - Individual Income Tax Statistics - 2015 ZIP Code Data (SOI), https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi

[3]	Yelp Fusion APIs, https://www.yelp.com/fusion